In [63]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math
#from sklearn.metrics import classification_report
import pyarrow as pa
import pyarrow.parquet as pq

`Importo dfs`

In [64]:
amazon_titles = pd.read_csv(r"C:/Users/rocio/OneDrive/Escritorio/Streaming_Platforms/Datasets/amazon_prime_titles.csv")
disney_titles = pd.read_csv(r"C:/Users/rocio/OneDrive/Escritorio/Streaming_Platforms/Datasets/disney_plus_titles.csv")
hulu_titles = pd.read_csv(r'C:/Users/rocio/OneDrive/Escritorio/Streaming_Platforms/Datasets/hulu_titles.csv')
netflix_titles= pd.read_csv(r"C:/Users/rocio/OneDrive/Escritorio/Streaming_Platforms/Datasets/netflix_titles.csv")

`Agrego una nueva columna para cada conjunto de datos para obtener la plataforma`

In [65]:

amazon_titles['platform'] = 'amazon'
disney_titles['platform'] = 'disney'
hulu_titles['platform'] = 'hulu'
netflix_titles['platform'] = 'netflix'

In [66]:
netflix_titles.head(4)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,platform
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",netflix
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",netflix
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,netflix
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",netflix


`Uno DFs`

In [67]:

platforms = pd.concat([disney_titles, netflix_titles, hulu_titles, amazon_titles], ignore_index=True)

In [68]:
platforms.head(4)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,platform
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!,disney
1,s2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,"November 26, 2021",1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...,disney
2,s3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,"November 26, 2021",2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.,disney
3,s4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",NaN,"November 26, 2021",2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!",disney


In [69]:
platforms.shape

(22998, 13)

In [70]:
platforms['id'] = platforms['platform'].str[0] + platforms['show_id'].astype(str)

In [71]:
platforms.head(4)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,platform,id
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!,disney,ds1
1,s2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,"November 26, 2021",1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...,disney,ds2
2,s3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,"November 26, 2021",2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.,disney,ds3
3,s4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",NaN,"November 26, 2021",2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!",disney,ds4


` Reemplazar los valores nulos del campo "rating" con el valor "G"`

In [72]:
platforms['rating'] = platforms['rating'].fillna('G')

In [73]:
platforms.head(20)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,platform,id
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!,disney,ds1
1,s2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,"November 26, 2021",1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...,disney,ds2
2,s3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,"November 26, 2021",2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.,disney,ds3
3,s4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",NaN,"November 26, 2021",2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!",disney,ds4
4,s5,TV Show,The Beatles: Get Back,NaN,"John Lennon, Paul McCartney, George Harrison, ...",NaN,"November 25, 2021",2021,G,1 Season,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...,disney,ds5
5,s6,Movie,Becoming Cousteau,Liz Garbus,"Jacques Yves Cousteau, Vincent Cassel",United States,"November 24, 2021",2021,PG-13,94 min,"Biographical, Documentary",An inside look at the legendary life of advent...,disney,ds6
6,s7,TV Show,Hawkeye,NaN,"Jeremy Renner, Hailee Steinfeld, Vera Farmiga,...",NaN,"November 24, 2021",2021,TV-14,1 Season,"Action-Adventure, Superhero",Clint Barton/Hawkeye must team up with skilled...,disney,ds7
7,s8,TV Show,Port Protection Alaska,NaN,"Gary Muehlberger, Mary Miller, Curly Leach, Sa...",United States,"November 24, 2021",2015,TV-14,2 Seasons,"Docuseries, Reality, Survival",Residents of Port Protection must combat volat...,disney,ds8
8,s9,TV Show,Secrets of the Zoo: Tampa,NaN,"Dr. Ray Ball, Dr. Lauren Smith, Chris Massaro,...",United States,"November 24, 2021",2019,TV-PG,2 Seasons,"Animals & Nature, Docuseries, Family",A day in the life at ZooTampa is anything but ...,disney,ds9
9,s10,Movie,A Muppets Christmas: Letters To Santa,Kirk R. Thatcher,"Steve Whitmire, Dave Goelz, Bill Barretta, Eri...",United States,"November 19, 2021",2008,G,45 min,"Comedy, Family, Musical",Celebrate the holiday season with all your fav...,disney,ds10


`Cambio el formato de las fecha a AAAA-mm-dd`

In [74]:
platforms['date_added'] = pd.to_datetime(platforms['date_added']).dt.strftime('%Y-%m-%d')

In [75]:
platforms.head(10)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,platform,id
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,2021-11-26,2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!,disney,ds1
1,s2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,2021-11-26,1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...,disney,ds2
2,s3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,2021-11-26,2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.,disney,ds3
3,s4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",NaN,2021-11-26,2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!",disney,ds4
4,s5,TV Show,The Beatles: Get Back,NaN,"John Lennon, Paul McCartney, George Harrison, ...",NaN,2021-11-25,2021,G,1 Season,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...,disney,ds5
5,s6,Movie,Becoming Cousteau,Liz Garbus,"Jacques Yves Cousteau, Vincent Cassel",United States,2021-11-24,2021,PG-13,94 min,"Biographical, Documentary",An inside look at the legendary life of advent...,disney,ds6
6,s7,TV Show,Hawkeye,NaN,"Jeremy Renner, Hailee Steinfeld, Vera Farmiga,...",NaN,2021-11-24,2021,TV-14,1 Season,"Action-Adventure, Superhero",Clint Barton/Hawkeye must team up with skilled...,disney,ds7
7,s8,TV Show,Port Protection Alaska,NaN,"Gary Muehlberger, Mary Miller, Curly Leach, Sa...",United States,2021-11-24,2015,TV-14,2 Seasons,"Docuseries, Reality, Survival",Residents of Port Protection must combat volat...,disney,ds8
8,s9,TV Show,Secrets of the Zoo: Tampa,NaN,"Dr. Ray Ball, Dr. Lauren Smith, Chris Massaro,...",United States,2021-11-24,2019,TV-PG,2 Seasons,"Animals & Nature, Docuseries, Family",A day in the life at ZooTampa is anything but ...,disney,ds9
9,s10,Movie,A Muppets Christmas: Letters To Santa,Kirk R. Thatcher,"Steve Whitmire, Dave Goelz, Bill Barretta, Eri...",United States,2021-11-19,2008,G,45 min,"Comedy, Family, Musical",Celebrate the holiday season with all your fav...,disney,ds10


`Paso todos los campos que esten en formato texto a string`

In [76]:
platforms = platforms.apply(lambda x: x.astype(str) if x.dtype == "object" else x)

In [77]:
platforms.head(4)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,platform,id
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",nan,2021-11-26,2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!,disney,ds1
1,s2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",nan,2021-11-26,1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...,disney,ds2
2,s3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,2021-11-26,2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.,disney,ds3
3,s4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",nan,2021-11-26,2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!",disney,ds4


In [78]:
platforms.dtypes

show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
platform        object
id              object
dtype: object

`Paso todos los campos que esten en string a minusculas`

In [79]:
platforms = platforms.applymap(lambda x: x.lower() if type(x) == str else x)

In [80]:
platforms.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,platform,id
0,s1,movie,duck the halls: a mickey mouse christmas special,"alonso ramirez ramos, dave wasson","chris diamantopoulos, tony anselmo, tress macn...",nan,2021-11-26,2016,tv-g,23 min,"animation, family",join mickey and the gang as they duck the halls!,disney,ds1
1,s2,movie,ernest saves christmas,john cherry,"jim varney, noelle parker, douglas seale",nan,2021-11-26,1988,pg,91 min,comedy,santa claus passes his magic bag to a new st. ...,disney,ds2
2,s3,movie,ice age: a mammoth christmas,karen disher,"raymond albert romano, john leguizamo, denis l...",united states,2021-11-26,2011,tv-g,23 min,"animation, comedy, family",sid the sloth is on santa's naughty list.,disney,ds3
3,s4,movie,the queen family singalong,hamish hamilton,"darren criss, adam lambert, derek hough, alexa...",nan,2021-11-26,2021,tv-pg,41 min,musical,"this is real life, not just fantasy!",disney,ds4
4,s5,tv show,the beatles: get back,nan,"john lennon, paul mccartney, george harrison, ...",nan,2021-11-25,2021,g,1 season,"docuseries, historical, music",a three-part documentary from peter jackson ca...,disney,ds5


`Separo el campo duration en dos campos: number_of_minutes y unit_of_measurement`

In [81]:
platforms[['duration_int', 'duration_type']] = platforms['duration'].str.split(' ', n=1, expand=True)

In [82]:
print(platforms['duration_type'])

0            min
1            min
2            min
3            min
4         season
          ...   
22993        min
22994    seasons
22995        min
22996     season
22997        min
Name: duration_type, Length: 22998, dtype: object


`unifico los valores de season y seasons en una sola columna season`

In [83]:
# crear una columna nueva con los valores corregidos
platforms['temp_season'] = platforms['duration_type'].replace({'seasons': 'season'})

In [84]:
# eliminar la columna original
platforms.drop('duration_type', axis=1, inplace=True)

In [85]:
# renombrar la columna "temp_season" a "duration_type"
platforms.rename(columns={'temp_season': 'duration_type'}, inplace=True)

In [86]:
platforms.head(4)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,platform,id,duration_int,duration_type
0,s1,movie,duck the halls: a mickey mouse christmas special,"alonso ramirez ramos, dave wasson","chris diamantopoulos, tony anselmo, tress macn...",nan,2021-11-26,2016,tv-g,23 min,"animation, family",join mickey and the gang as they duck the halls!,disney,ds1,23,min
1,s2,movie,ernest saves christmas,john cherry,"jim varney, noelle parker, douglas seale",nan,2021-11-26,1988,pg,91 min,comedy,santa claus passes his magic bag to a new st. ...,disney,ds2,91,min
2,s3,movie,ice age: a mammoth christmas,karen disher,"raymond albert romano, john leguizamo, denis l...",united states,2021-11-26,2011,tv-g,23 min,"animation, comedy, family",sid the sloth is on santa's naughty list.,disney,ds3,23,min
3,s4,movie,the queen family singalong,hamish hamilton,"darren criss, adam lambert, derek hough, alexa...",nan,2021-11-26,2021,tv-pg,41 min,musical,"this is real life, not just fantasy!",disney,ds4,41,min


In [87]:
print(platforms['duration_type'])

0           min
1           min
2           min
3           min
4        season
          ...  
22993       min
22994    season
22995       min
22996    season
22997       min
Name: duration_type, Length: 22998, dtype: object


`Renombro release_year por year`

In [88]:
platforms = platforms.rename(columns={'release_year': 'year'})

In [89]:
platforms.head(25)

,show_id,type,title,director,cast,country,date_added,year,rating,duration,listed_in,description,platform,id,duration_int,duration_type
0,s1,movie,duck the halls: a mickey mouse christmas special,"alonso ramirez ramos, dave wasson","chris diamantopoulos, tony anselmo, tress macn...",nan,2021-11-26,2016,tv-g,23 min,"animation, family",join mickey and the gang as they duck the halls!,disney,ds1,23,min
1,s2,movie,ernest saves christmas,john cherry,"jim varney, noelle parker, douglas seale",nan,2021-11-26,1988,pg,91 min,comedy,santa claus passes his magic bag to a new st. ...,disney,ds2,91,min
2,s3,movie,ice age: a mammoth christmas,karen disher,"raymond albert romano, john leguizamo, denis l...",united states,2021-11-26,2011,tv-g,23 min,"animation, comedy, family",sid the sloth is on santa's naughty list.,disney,ds3,23,min
3,s4,movie,the queen family singalong,hamish hamilton,"darren criss, adam lambert, derek hough, alexa...",nan,2021-11-26,2021,tv-pg,41 min,musical,"this is real life, not just fantasy!",disney,ds4,41,min
4,s5,tv show,the beatles: get back,nan,"john lennon, paul mccartney, george harrison, ...",nan,2021-11-25,2021,g,1 season,"docuseries, historical, music",a three-part documentary from peter jackson ca...,disney,ds5,1,season
5,s6,movie,becoming cousteau,liz garbus,"jacques yves cousteau, vincent cassel",united states,2021-11-24,2021,pg-13,94 min,"biographical, documentary",an inside look at the legendary life of advent...,disney,ds6,94,min
6,s7,tv show,hawkeye,nan,"jeremy renner, hailee steinfeld, vera farmiga,...",nan,2021-11-24,2021,tv-14,1 season,"action-adventure, superhero",clint barton/hawkeye must team up with skilled...,disney,ds7,1,season
7,s8,tv show,port protection alaska,nan,"gary muehlberger, mary miller, curly leach, sa...",united states,2021-11-24,2015,tv-14,2 seasons,"docuseries, reality, survival",residents of port protection must combat volat...,disney,ds8,2,season
8,s9,tv show,secrets of the zoo: tampa,nan,"dr. ray ball, dr. lauren smith, chris massaro,...",united states,2021-11-24,2019,tv-pg,2 seasons,"animals & nature, docuseries, family",a day in the life at zootampa is anything but ...,disney,ds9,2,season
9,s10,movie,a muppets christmas: letters to santa,kirk r. thatcher,"steve whitmire, dave goelz, bill barretta, eri...",united states,2021-11-19,2008,g,45 min,"comedy, family, musical",celebrate the holiday season with all your fav...,disney,ds10,45,min


`Importo los 8 dataframes de ratings`

In [90]:

df1 = pd.read_csv(r'C:/Users/rocio/OneDrive/Escritorio/Streaming_Platforms/Ratings/1.csv')
df2 = pd.read_csv(r'C:/Users/rocio/OneDrive/Escritorio/Streaming_Platforms/Ratings/2.csv')
df3 = pd.read_csv(r'C:/Users/rocio/OneDrive/Escritorio/Streaming_Platforms/Ratings/3.csv')
df4 = pd.read_csv(r'C:/Users/rocio/OneDrive/Escritorio/Streaming_Platforms/Ratings/4.csv')
df5 = pd.read_csv(r'C:/Users/rocio/OneDrive/Escritorio/Streaming_Platforms/Ratings/5.csv')
df6 = pd.read_csv(r'C:/Users/rocio/OneDrive/Escritorio/Streaming_Platforms/Ratings/6.csv')
df7 = pd.read_csv(r'C:/Users/rocio/OneDrive/Escritorio/Streaming_Platforms/Ratings/7.csv')
df8 = pd.read_csv(r'C:/Users/rocio/OneDrive/Escritorio/Streaming_Platforms/Ratings/8.csv')

In [91]:
df1.head(4)

,userId,rating,timestamp,movieId
0,1,1.0,1425941529,as680
1,1,4.5,1425942435,ns2186
2,1,5.0,1425941523,hs2381
3,1,5.0,1425941546,ns3663


In [92]:
df1.info

<bound method DataFrame.info of          userId  rating   timestamp movieId
0             1     1.0  1425941529   as680
1             1     4.5  1425942435  ns2186
2             1     5.0  1425941523  hs2381
3             1     5.0  1425941546  ns3663
4             1     5.0  1425941556  as9500
...         ...     ...         ...     ...
1499995   15424     4.0  1370209959  as8698
1499996   15424     3.5  1369586045  as3663
1499997   15424     3.0  1372541792  as1325
1499998   15424     3.5  1372541811  ns4446
1499999   15424     3.5  1369606242  hs2569

[1500000 rows x 4 columns]>

`Los uno en un solo dataframe llamado Ratings`

In [93]:
ratings = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], ignore_index=True)

In [94]:
ratings.head(4)

,userId,rating,timestamp,movieId
0,1,1.0,1425941529,as680
1,1,4.5,1425942435,ns2186
2,1,5.0,1425941523,hs2381
3,1,5.0,1425941546,ns3663


In [95]:
ratings.info

<bound method DataFrame.info of           userId  rating   timestamp movieId
0              1     1.0  1425941529   as680
1              1     4.5  1425942435  ns2186
2              1     5.0  1425941523  hs2381
3              1     5.0  1425941546  ns3663
4              1     5.0  1425941556  as9500
...          ...     ...         ...     ...
11024284  124380     4.5  1196786159  ns5272
11024285  124380     2.5  1196786030  ns5492
11024286  124380     3.5  1196785679   hs305
11024287  124380     4.5  1196787089  ns7881
11024288  124380     1.5  1196785847   as883

[11024289 rows x 4 columns]>

`Cambio el nombre de rating por score`

In [96]:
ratings = ratings.rename(columns={'rating': 'score'})

`Cambio el nombre de movieId por id`

In [97]:
ratings = ratings.rename(columns={'movieId': 'id'})

In [98]:
ratings.columns

Index(['userId', 'score', 'timestamp', 'id'], dtype='object')

`Transformo los valores nulos de score por "G"`

In [99]:
ratings['score'] = ratings['score'].fillna('G')


`No hay formatos de fecha que cambiar, ahora paso todos los valores que estren en string a minusculas`

In [100]:
ratings = ratings.applymap(lambda x: x.lower() if type(x) == str else x)


In [101]:
ratings.head(4)

,userId,score,timestamp,id
0,1,1.0,1425941529,as680
1,1,4.5,1425942435,ns2186
2,1,5.0,1425941523,hs2381
3,1,5.0,1425941546,ns3663


`Realizo un promedio de la columna score, agrupada por promedio.`

In [102]:
promedio_score = ratings.groupby('id')['score'].mean().round(2)

`Realizo un dataframe pequeño con los promedios con los respectivos id`


In [103]:
score_mean = pd.DataFrame({'id': promedio_score.index, 'score_mean': promedio_score.values})

`Visualizo el pequeño dataframe(tienen que quedar solo dos columnas con el id y con el promedio de score)`

In [104]:
score_mean.head()

,id,score_mean
0,as1,3.47
1,as10,3.44
2,as100,3.61
3,as1000,3.56
4,as1001,3.59


`Creo un nuevo dataframe donde junto el dataframe de Platforms + score_mean`

In [105]:
platforms_and_score = pd.merge(platforms, score_mean, on='id', how='left')

In [106]:
platforms_and_score.head(4)

,show_id,type,title,director,cast,country,date_added,year,rating,duration,listed_in,description,platform,id,duration_int,duration_type,score_mean
0,s1,movie,duck the halls: a mickey mouse christmas special,"alonso ramirez ramos, dave wasson","chris diamantopoulos, tony anselmo, tress macn...",nan,2021-11-26,2016,tv-g,23 min,"animation, family",join mickey and the gang as they duck the halls!,disney,ds1,23,min,3.55
1,s2,movie,ernest saves christmas,john cherry,"jim varney, noelle parker, douglas seale",nan,2021-11-26,1988,pg,91 min,comedy,santa claus passes his magic bag to a new st. ...,disney,ds2,91,min,3.49
2,s3,movie,ice age: a mammoth christmas,karen disher,"raymond albert romano, john leguizamo, denis l...",united states,2021-11-26,2011,tv-g,23 min,"animation, comedy, family",sid the sloth is on santa's naughty list.,disney,ds3,23,min,3.56
3,s4,movie,the queen family singalong,hamish hamilton,"darren criss, adam lambert, derek hough, alexa...",nan,2021-11-26,2021,tv-pg,41 min,musical,"this is real life, not just fantasy!",disney,ds4,41,min,3.51


In [107]:
platforms_and_score = platforms_and_score.rename(columns={'score_mean': 'scored'})

In [108]:
platforms_and_score['year'] = platforms_and_score['year'].astype('int')

In [109]:
platforms_and_score.head(4)

,show_id,type,title,director,cast,country,date_added,year,rating,duration,listed_in,description,platform,id,duration_int,duration_type,scored
0,s1,movie,duck the halls: a mickey mouse christmas special,"alonso ramirez ramos, dave wasson","chris diamantopoulos, tony anselmo, tress macn...",nan,2021-11-26,2016,tv-g,23 min,"animation, family",join mickey and the gang as they duck the halls!,disney,ds1,23,min,3.55
1,s2,movie,ernest saves christmas,john cherry,"jim varney, noelle parker, douglas seale",nan,2021-11-26,1988,pg,91 min,comedy,santa claus passes his magic bag to a new st. ...,disney,ds2,91,min,3.49
2,s3,movie,ice age: a mammoth christmas,karen disher,"raymond albert romano, john leguizamo, denis l...",united states,2021-11-26,2011,tv-g,23 min,"animation, comedy, family",sid the sloth is on santa's naughty list.,disney,ds3,23,min,3.56
3,s4,movie,the queen family singalong,hamish hamilton,"darren criss, adam lambert, derek hough, alexa...",nan,2021-11-26,2021,tv-pg,41 min,musical,"this is real life, not just fantasy!",disney,ds4,41,min,3.51


In [110]:
platforms_and_score.year

0        2016
1        1988
2        2011
3        2021
4        2021
         ... 
22993    1940
22994    2018
22995    2008
22996    2021
22997    2010
Name: year, Length: 22998, dtype: int32

`Este último dataset llamado platforms_and_score es el que voy a utilizar para hacer las 4 consultas a la API`

In [111]:
platforms_and_score.shape

(22998, 17)

In [112]:
platforms_and_score.to_csv('platforms_and_score.csv', index=False)

In [114]:
platforms_and_score['year'].dtype

dtype('int32')

In [113]:
#print(year, platform, duration_type)

NameError: name 'year' is not defined

`Junto plataforms_and_score con rating, que va a contener todos los aproximadamente 11.000.000 datos `

In [ ]:
# Eliminamos la columna 'scored' del dataframe 'platforms_and_score'
platforms_and_score = platforms_and_score.drop('scored', axis=1)

In [ ]:
platforms_and_score.head(4)

,show_id,type,title,director,cast,country,date_added,year,rating,duration,listed_in,description,platform,id,duration_int,duration_type
0,s1,movie,duck the halls: a mickey mouse christmas special,"alonso ramirez ramos, dave wasson","chris diamantopoulos, tony anselmo, tress macn...",nan,2021-11-26,2016,tv-g,23 min,"animation, family",join mickey and the gang as they duck the halls!,disney,ds1,23,min
1,s2,movie,ernest saves christmas,john cherry,"jim varney, noelle parker, douglas seale",nan,2021-11-26,1988,pg,91 min,comedy,santa claus passes his magic bag to a new st. ...,disney,ds2,91,min
2,s3,movie,ice age: a mammoth christmas,karen disher,"raymond albert romano, john leguizamo, denis l...",united states,2021-11-26,2011,tv-g,23 min,"animation, comedy, family",sid the sloth is on santa's naughty list.,disney,ds3,23,min
3,s4,movie,the queen family singalong,hamish hamilton,"darren criss, adam lambert, derek hough, alexa...",nan,2021-11-26,2021,tv-pg,41 min,musical,"this is real life, not just fantasy!",disney,ds4,41,min


In [ ]:
# Unimos los dataframes en base a la columna 'id'
movies_and_series = pd.merge(platforms_and_score, ratings, on='id')

In [ ]:
movies_and_series.head(4)

,show_id,type,title,director,cast,country,date_added,year,rating,duration,listed_in,description,platform,id,duration_int,duration_type,userId,score,timestamp
0,s1,movie,duck the halls: a mickey mouse christmas special,"alonso ramirez ramos, dave wasson","chris diamantopoulos, tony anselmo, tress macn...",nan,2021-11-26,2016,tv-g,23 min,"animation, family",join mickey and the gang as they duck the halls!,disney,ds1,23,min,101,4.0,850689925
1,s1,movie,duck the halls: a mickey mouse christmas special,"alonso ramirez ramos, dave wasson","chris diamantopoulos, tony anselmo, tress macn...",nan,2021-11-26,2016,tv-g,23 min,"animation, family",join mickey and the gang as they duck the halls!,disney,ds1,23,min,251,2.0,1055006120
2,s1,movie,duck the halls: a mickey mouse christmas special,"alonso ramirez ramos, dave wasson","chris diamantopoulos, tony anselmo, tress macn...",nan,2021-11-26,2016,tv-g,23 min,"animation, family",join mickey and the gang as they duck the halls!,disney,ds1,23,min,400,3.0,850385998
3,s1,movie,duck the halls: a mickey mouse christmas special,"alonso ramirez ramos, dave wasson","chris diamantopoulos, tony anselmo, tress macn...",nan,2021-11-26,2016,tv-g,23 min,"animation, family",join mickey and the gang as they duck the halls!,disney,ds1,23,min,1572,3.5,1470335824


In [ ]:
num_registros = movies_and_series.shape[0]

In [ ]:
# Imprimimos el número de registros
print(f'El dataframe resultante tiene {num_registros} registros.')

El dataframe resultante tiene 11024289 registros.


`Paso ese df movies and series a formato parquet`

In [ ]:
tabla = pa.Table.from_pandas(movies_and_series)


ArrowMemoryError: realloc of size 1073741824 failed

In [ ]:
pq.write_table(tabla, 'movies_and_series.parquet')